In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import StandardScaler
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error,accuracy_score,f1_score
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import mean_squared_error,log_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
df_user_data_train = pd.read_csv("train/user_data.csv")
df_train_submission = pd.read_csv("train/train_submissions.csv")
df_problem_data_train = pd.read_csv("train/problem_data.csv")
df_sample_sub = pd.read_csv("sample_submissions_wbscxqU.csv")
df_test_sub = pd.read_csv("test_submissions_NeDLEvX.csv")

In [3]:
from keras import backend as K

def mcor(y_true, y_pred):
     #matthews_correlation
     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
     y_pred_neg = 1 - y_pred_pos
 
 
     y_pos = K.round(K.clip(y_true, 0, 1))
     y_neg = 1 - y_pos
 
 
     tp = K.sum(y_pos * y_pred_pos)
     tn = K.sum(y_neg * y_pred_neg)
 
 
     fp = K.sum(y_neg * y_pred_pos)
     fn = K.sum(y_pos * y_pred_neg)
 
 
     numerator = (tp * tn - fp * fn)
     denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
 
 
     return numerator / (denominator + K.epsilon())




def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


In [4]:
df_problem_data_train.drop(['tags'],axis=1,inplace=True)
#df_problem_data_train.points.fillna(0,inplace=True)
df_problem_data_train.level_type.fillna('N',inplace=True)
df_user_data_train.country.fillna('Missing',inplace=True)

def postive_only(x):
    if(x<0):
        return(-1*x)
    else:
        return(x)

def get_points(x1):
    if(x1=="A"):
        return(500)
    elif(x1=='B'):
        return(1000)
    elif(x1=='C'):
        return(1500)
    elif(x1=='D'):
        return(2000)
    elif(x1=='E'):
        return(2500)
    elif(x1=='F'):
        return(3000)
    elif(x1=='G'):
        return(3500)
    elif(x1=='H'):
        return(4000)
    else:
        return(1000)

df_problem_data_train['points']=df_problem_data_train['level_type'].map(lambda x: get_points(x))
df_user_data_train['submission_count']=df_user_data_train['submission_count'].map(lambda x: postive_only(x))
df_user_data_train['problem_solved']=df_user_data_train['problem_solved'].map(lambda x: postive_only(x))
df_user_data_train['contribution']=df_user_data_train['contribution'].map(lambda x: postive_only(x))
df_user_data_train['follower_count']=df_user_data_train['follower_count'].map(lambda x: postive_only(x))
df_user_data_train['last_online_time_seconds']=df_user_data_train['last_online_time_seconds'].map(lambda x: postive_only(x))
df_user_data_train['max_rating']=df_user_data_train['max_rating'].map(lambda x: postive_only(x))
df_user_data_train['registration_time_seconds']=df_user_data_train['registration_time_seconds'].map(lambda x: postive_only(x))

#from sklearn.preprocessing import LabelEncoder
#lb = LabelEncoder()
#df_user_data_train['country'] = lb.fit_transform(df_user_data_train['country'])
#df_user_data_train['rank'] = lb.fit_transform(df_user_data_train['rank'])
#df_problem_data_train['level_type'] = lb.fit_transform(df_problem_data_train['level_type'])


In [5]:
df_user_data_train['problem_sub_ratio'] = df_user_data_train['problem_solved'] / df_user_data_train['submission_count']

df_user_data_train['last_online_time_seconds'] = df_user_data_train.last_online_time_seconds.max() - df_user_data_train['last_online_time_seconds']
df_user_data_train['last_online_time_minutes'] = df_user_data_train['last_online_time_seconds'] / 60
df_user_data_train['last_online_time_hours'] = df_user_data_train['last_online_time_seconds'] / (60*60)
df_user_data_train['last_online_time_days'] = df_user_data_train['last_online_time_seconds'] / (60*60*24)
df_user_data_train['last_online_time_months'] = df_user_data_train['last_online_time_seconds'] / (60*60*24*30)
df_user_data_train['last_online_time_years'] = df_user_data_train['last_online_time_seconds'] / (60*60*24*365)

df_user_data_train['registration_time_seconds'] = df_user_data_train.registration_time_seconds.max() - df_user_data_train['registration_time_seconds']
df_user_data_train['registration_time_minutes'] = df_user_data_train['registration_time_seconds'] / 60
df_user_data_train['registration_time_hours'] = df_user_data_train['registration_time_seconds'] / (60*60)
df_user_data_train['registration_time_days'] = df_user_data_train['registration_time_seconds'] / (60*60*24)
df_user_data_train['registration_time_months'] = df_user_data_train['registration_time_seconds'] / (60*60*24*30)
df_user_data_train['registration_time_years'] = df_user_data_train['registration_time_seconds'] / (60*60*24*365)

df_user_data_train['last_vs_reg_time_ratio'] = df_user_data_train['last_online_time_seconds']/(1+df_user_data_train['registration_time_seconds'])

df_user_data_train['max_rating_current_ratio'] = df_user_data_train['max_rating']/ (1+df_user_data_train['rating'])
df_user_data_train['current_rating_per_registration_time'] = df_user_data_train['rating']/(1+df_user_data_train['registration_time_days'])
df_user_data_train['Follower_vs_Rating'] = df_user_data_train['follower_count']/(1+df_user_data_train['rating'])
df_user_data_train['Rating_mul_Follower'] = df_user_data_train['rating'] * df_user_data_train['follower_count']
df_user_data_train['follower_vs_Contribution'] = df_user_data_train['follower_count']/(0.1+df_user_data_train['contribution'])



df_problem_data_train = pd.get_dummies(df_problem_data_train,columns=['level_type'])
df_user_data_train = pd.get_dummies(df_user_data_train,columns=['country','rank'])

In [6]:
df_train  = pd.merge(df_train_submission,df_user_data_train, how='left', on='user_id')
df_train = pd.merge(df_train,df_problem_data_train,how='left',on='problem_id')

df_test  = pd.merge(df_test_sub,df_user_data_train, how='left', on='user_id')
df_test = pd.merge(df_test,df_problem_data_train,how='left',on='problem_id')

X = df_train.drop(['attempts_range','user_id','problem_id'],axis=1).values
y = df_train.attempts_range.values

X_test = df_test.drop(['ID','user_id','problem_id'],axis=1).values

ss = StandardScaler()
X = ss.fit_transform(X)
X_test_sub = ss.transform(X_test)

dummy_y = np_utils.to_categorical(y)

#X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=42)

In [7]:
clf_RF =RandomForestClassifier()
y_pred_cross = cross_val_predict(clf_RF,X,y,cv=8,n_jobs=-1)
print(f1_score(y,y_pred_cross,average="weighted"))
clf_RF = RandomForestClassifier()
clf_RF.fit(X,y)
y_pred = clf_RF.predict(X_test_sub)
df_train['y_pred'] = y_pred_cross
df_test['y_pred'] = y_pred
X = df_train.drop(['attempts_range','user_id','problem_id'],axis=1).values
y = df_train.attempts_range.values

X_test_sub = df_test.drop(['ID','user_id','problem_id'],axis=1).values

ss = StandardScaler()
X = ss.fit_transform(X)
X_test_sub = ss.transform(X_test_sub)
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=42)

0.444180639669


In [8]:
dummy_y

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

In [9]:
X_train.shape
X_test.shape

(51248, 125)

In [10]:
model = Sequential()
model.add(Dense(1200, input_dim=X.shape[1], activation='relu'))
model.add(Dense(640, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(dummy_y.shape[1], activation='softmax'))

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])

In [12]:

model.fit(X_train, y_train, epochs=500, batch_size=10000,validation_data=(X_test, y_test))


Train on 104047 samples, validate on 51248 samples
Epoch 1/500
104047/104047 [==============================] - 1s - loss: 1.3862 - f1: 0.2900 - val_loss: 1.2007 - val_f1: 0.4011
Epoch 2/500
104047/104047 [==============================] - 0s - loss: 1.1826 - f1: 0.4368 - val_loss: 1.1665 - val_f1: 0.4497
Epoch 3/500
104047/104047 [==============================] - 0s - loss: 1.1555 - f1: 0.4512 - val_loss: 1.1555 - val_f1: 0.4571
Epoch 4/500
104047/104047 [==============================] - 0s - loss: 1.1462 - f1: 0.4585 - val_loss: 1.1516 - val_f1: 0.4591
Epoch 5/500
104047/104047 [==============================] - 0s - loss: 1.1421 - f1: 0.4561 - val_loss: 1.1501 - val_f1: 0.4712
Epoch 6/500
104047/104047 [==============================] - 0s - loss: 1.1399 - f1: 0.4578 - val_loss: 1.1493 - val_f1: 0.4531
Epoch 7/500
104047/104047 [==============================] - 0s - loss: 1.1371 - f1: 0.4558 - val_loss: 1.1486 - val_f1: 0.4601
Epoch 8/500
104047/104047 [==========================

104047/104047 [==============================] - 0s - loss: 1.0215 - f1: 0.5236 - val_loss: 1.2355 - val_f1: 0.4628
Epoch 65/500
104047/104047 [==============================] - 0s - loss: 1.0200 - f1: 0.5270 - val_loss: 1.2372 - val_f1: 0.4575
Epoch 66/500
104047/104047 [==============================] - 0s - loss: 1.0184 - f1: 0.5271 - val_loss: 1.2436 - val_f1: 0.4455
Epoch 67/500
104047/104047 [==============================] - 0s - loss: 1.0138 - f1: 0.5279 - val_loss: 1.2467 - val_f1: 0.4609
Epoch 68/500
104047/104047 [==============================] - 0s - loss: 1.0152 - f1: 0.5303 - val_loss: 1.2446 - val_f1: 0.4419
Epoch 69/500
104047/104047 [==============================] - 0s - loss: 1.0122 - f1: 0.5283 - val_loss: 1.2497 - val_f1: 0.4509
Epoch 70/500
104047/104047 [==============================] - 0s - loss: 1.0092 - f1: 0.5313 - val_loss: 1.2595 - val_f1: 0.4707
Epoch 71/500
104047/104047 [==============================] - 0s - loss: 1.0128 - f1: 0.5309 - val_loss: 1.250

104047/104047 [==============================] - 1s - loss: 0.9204 - f1: 0.5784 - val_loss: 1.4041 - val_f1: 0.4734
Epoch 128/500
104047/104047 [==============================] - 1s - loss: 0.9161 - f1: 0.5787 - val_loss: 1.4242 - val_f1: 0.5047
Epoch 129/500
104047/104047 [==============================] - 1s - loss: 0.9158 - f1: 0.5775 - val_loss: 1.4289 - val_f1: 0.4706
Epoch 130/500
104047/104047 [==============================] - 1s - loss: 0.9166 - f1: 0.5796 - val_loss: 1.4179 - val_f1: 0.4839
Epoch 131/500
104047/104047 [==============================] - 1s - loss: 0.9147 - f1: 0.5807 - val_loss: 1.4189 - val_f1: 0.4714
Epoch 132/500
104047/104047 [==============================] - 1s - loss: 0.9085 - f1: 0.5811 - val_loss: 1.4289 - val_f1: 0.4799
Epoch 133/500
104047/104047 [==============================] - 0s - loss: 0.9049 - f1: 0.5847 - val_loss: 1.4302 - val_f1: 0.4777
Epoch 134/500
104047/104047 [==============================] - 1s - loss: 0.9069 - f1: 0.5847 - val_loss

104047/104047 [==============================] - 1s - loss: 0.8639 - f1: 0.6019 - val_loss: 1.5468 - val_f1: 0.4887
Epoch 191/500
104047/104047 [==============================] - 1s - loss: 0.8649 - f1: 0.6012 - val_loss: 1.5598 - val_f1: 0.5091
Epoch 192/500
104047/104047 [==============================] - 1s - loss: 0.8705 - f1: 0.5985 - val_loss: 1.5574 - val_f1: 0.4568
Epoch 193/500
104047/104047 [==============================] - 1s - loss: 0.8667 - f1: 0.5982 - val_loss: 1.5557 - val_f1: 0.4937
Epoch 194/500
104047/104047 [==============================] - 1s - loss: 0.8626 - f1: 0.6016 - val_loss: 1.5457 - val_f1: 0.4942
Epoch 195/500
104047/104047 [==============================] - 1s - loss: 0.8638 - f1: 0.6023 - val_loss: 1.5437 - val_f1: 0.4719
Epoch 196/500
104047/104047 [==============================] - 1s - loss: 0.8635 - f1: 0.5996 - val_loss: 1.5537 - val_f1: 0.4969
Epoch 197/500
104047/104047 [==============================] - 1s - loss: 0.8612 - f1: 0.6039 - val_loss

104047/104047 [==============================] - 0s - loss: 0.8395 - f1: 0.6112 - val_loss: 1.6344 - val_f1: 0.4693
Epoch 254/500
104047/104047 [==============================] - 0s - loss: 0.8399 - f1: 0.6099 - val_loss: 1.6352 - val_f1: 0.4910
Epoch 255/500
104047/104047 [==============================] - 0s - loss: 0.8400 - f1: 0.6122 - val_loss: 1.6378 - val_f1: 0.4542
Epoch 256/500
104047/104047 [==============================] - 0s - loss: 0.8401 - f1: 0.6106 - val_loss: 1.6413 - val_f1: 0.4642
Epoch 257/500
104047/104047 [==============================] - 0s - loss: 0.8359 - f1: 0.6107 - val_loss: 1.6368 - val_f1: 0.4868
Epoch 258/500
104047/104047 [==============================] - 0s - loss: 0.8394 - f1: 0.6120 - val_loss: 1.6312 - val_f1: 0.5019
Epoch 259/500
104047/104047 [==============================] - 0s - loss: 0.8343 - f1: 0.6119 - val_loss: 1.6342 - val_f1: 0.4788
Epoch 260/500
104047/104047 [==============================] - 0s - loss: 0.8331 - f1: 0.6137 - val_loss

104047/104047 [==============================] - 1s - loss: 0.8190 - f1: 0.6188 - val_loss: 1.6974 - val_f1: 0.4914
Epoch 317/500
104047/104047 [==============================] - 1s - loss: 0.8212 - f1: 0.6176 - val_loss: 1.6942 - val_f1: 0.4904
Epoch 318/500
104047/104047 [==============================] - 1s - loss: 0.8179 - f1: 0.6204 - val_loss: 1.6960 - val_f1: 0.4813
Epoch 319/500
104047/104047 [==============================] - 1s - loss: 0.8207 - f1: 0.6176 - val_loss: 1.7009 - val_f1: 0.4935.61
Epoch 320/500
104047/104047 [==============================] - 1s - loss: 0.8210 - f1: 0.6187 - val_loss: 1.6852 - val_f1: 0.4895
Epoch 321/500
104047/104047 [==============================] - 1s - loss: 0.8188 - f1: 0.6181 - val_loss: 1.7024 - val_f1: 0.4991
Epoch 322/500
104047/104047 [==============================] - 1s - loss: 0.8208 - f1: 0.6178 - val_loss: 1.7320 - val_f1: 0.4638
Epoch 323/500
104047/104047 [==============================] - 1s - loss: 0.8261 - f1: 0.6134 - val_l

104047/104047 [==============================] - 0s - loss: 0.8135 - f1: 0.6206 - val_loss: 1.7329 - val_f1: 0.4580
Epoch 380/500
104047/104047 [==============================] - 0s - loss: 0.8101 - f1: 0.6224 - val_loss: 1.7259 - val_f1: 0.4638
Epoch 381/500
104047/104047 [==============================] - 0s - loss: 0.8075 - f1: 0.6215 - val_loss: 1.7430 - val_f1: 0.4783
Epoch 382/500
104047/104047 [==============================] - 0s - loss: 0.8058 - f1: 0.6229 - val_loss: 1.7437 - val_f1: 0.4867
Epoch 383/500
104047/104047 [==============================] - 0s - loss: 0.8046 - f1: 0.6231 - val_loss: 1.7500 - val_f1: 0.4930
Epoch 384/500
104047/104047 [==============================] - 0s - loss: 0.8045 - f1: 0.6228 - val_loss: 1.7616 - val_f1: 0.4964
Epoch 385/500
104047/104047 [==============================] - 0s - loss: 0.8045 - f1: 0.6234 - val_loss: 1.7435 - val_f1: 0.4711
Epoch 386/500
104047/104047 [==============================] - 0s - loss: 0.8053 - f1: 0.6223 - val_loss

104047/104047 [==============================] - 1s - loss: 0.8026 - f1: 0.6252 - val_loss: 1.7746 - val_f1: 0.4805
Epoch 443/500
104047/104047 [==============================] - 1s - loss: 0.8003 - f1: 0.6250 - val_loss: 1.7653 - val_f1: 0.4733
Epoch 444/500
104047/104047 [==============================] - 1s - loss: 0.7998 - f1: 0.6249 - val_loss: 1.7778 - val_f1: 0.4884
Epoch 445/500
104047/104047 [==============================] - 1s - loss: 0.8025 - f1: 0.6227 - val_loss: 1.7694 - val_f1: 0.5012
Epoch 446/500
104047/104047 [==============================] - 0s - loss: 0.8001 - f1: 0.6237 - val_loss: 1.7765 - val_f1: 0.4807
Epoch 447/500
104047/104047 [==============================] - 0s - loss: 0.7986 - f1: 0.6256 - val_loss: 1.7753 - val_f1: 0.4851
Epoch 448/500
104047/104047 [==============================] - 0s - loss: 0.7985 - f1: 0.6245 - val_loss: 1.7852 - val_f1: 0.4944
Epoch 449/500
104047/104047 [==============================] - 0s - loss: 0.7978 - f1: 0.6260 - val_loss

In [13]:
y_pred = model.predict(X_test_sub,batch_size=10000)

In [14]:
y_pred_val = y_pred.argmax(1)


In [15]:
df_sample_sub['ID'] = df_test.ID
df_sample_sub['attempts_range'] = y_pred_val
df_sample_sub.to_csv("Keras_with_Basic__FE.csv",index=False)